In [6]:
import os
import pandas as pd
import concurrent.futures
from scipy.io import wavfile
from scipy.spatial.distance import cosine
import numpy as np
from python_speech_features import mfcc
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import time  

# Function to read CSV data
def read_csv(file_path):
    return pd.read_csv(file_path)

# Function to load audio from file
def load_audio(audio_file):
    return wavfile.read(audio_file)

# Function to match IDs with audio file names
def match_audio_files(csv_data, audio_dir):
    id_to_audio = {}
    for index, row in csv_data.iterrows():
        audio_file = os.path.join(audio_dir, f"{row['id']}.wav")
        if os.path.exists(audio_file):
            id_to_audio[row['id']] = audio_file
    return id_to_audio

# Function to compare new audio with existing audio
def compare_audio(new_audio_path, existing_audios):
    if not os.path.exists(new_audio_path):
        return None
    
    new_sr, new_audio = wavfile.read(new_audio_path)
    min_distance = float('inf')
    closest_match_id = None

    # Iterate over existing audios
    for id_, (existing_sr, existing_audio) in existing_audios.items():
        # Check if sample rates are compatible
        if new_sr == existing_sr:
            # Calculate the length of the shorter audio
            min_length = min(len(new_audio), len(existing_audio))
            # Sample the new audio to match the length of the existing audio
            sampled_new_audio = new_audio[:min_length]
            # Calculate cosine distance between the sampled new audio and existing audio
            distance = cosine(sampled_new_audio.flatten(), existing_audio.flatten()[:min_length])
            # Update closest match if necessary
            if distance < min_distance:
                min_distance = distance
                closest_match_id = id_
    
    return closest_match_id

# Function to extract audio features
def extract_audio_features(audio_data, sample_rate):
    # Extract MFCC features from audio
    mfcc_features = mfcc(audio_data, sample_rate)
    # Calculate mean of MFCC features across frames
    mean_mfcc_features = np.mean(mfcc_features, axis=0)
    return mean_mfcc_features

# Function to train SVM model
def train_model(X_train, y_train):
    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    
    # Pipeline for feature scaling and SVM
    svm_model = Pipeline([
        ("scaler", StandardScaler()),
        ("svm", SVC(kernel='rbf', C=1.0, gamma='scale'))
    ])
    
    # Train the model
    svm_model.fit(X_train, y_train)
    
    # Evaluate the model if needed
    train_accuracy = svm_model.score(X_train, y_train)
    test_accuracy = svm_model.score(X_test, y_test)
    print("Train Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    
    return svm_model

# Function to predict species from new audio
def predict_species(new_audio_path, model):
    # Read new audio
    sample_rate, new_audio = wavfile.read(new_audio_path)
    # Extract audio features
    audio_features = extract_audio_features(new_audio, sample_rate)
    # Predict with the model
    predicted_species = model.predict([audio_features])
    print(predicted_species)
    return predicted_species[0]

# Function to update CSV data with prediction result
def update_csv(csv_file, row_id, species):
    data = pd.read_csv(csv_file)
    data.loc[data['id'] == row_id, 'species'] = species
    data.to_csv(csv_file, index=False)

# Main function

# Main function
def main():
    start_time = time.time()
    
    # CSV file location and audio directory
    csv_file = r"E:\KERJA\spudniklab\InsecstopProjeck\data\metadata\data.csv"
    audio_dir = r"E:\KERJA\spudniklab\InsecstopProjeck\data\audio"

    # Read data from CSV
    csv_data = read_csv(csv_file)

    # Match IDs with audio file names
    id_to_audio = match_audio_files(csv_data, audio_dir)

    # Process audio in parallel
    with concurrent.futures.ProcessPoolExecutor() as executor:
        audio_data = {id_: load_audio(audio_file) for id_, audio_file in id_to_audio.items()}

    # Extract features from audio data
    X_train = []
    y_train = []
    for id_, (sample_rate, audio) in audio_data.items():
        features = extract_audio_features(audio, sample_rate)
        X_train.append(features)
        y_train.append(id_)

    # Train SVM model
    svm_model = train_model(X_train, y_train)

    # Example of new audio
    new_audio_path = r"E:\KERJA\spudniklab\InsecstopProjeck\upload\zy.wav"

    # Predict species for new audio
    predicted_species = predict_species(new_audio_path, svm_model)

    # Update CSV with prediction result
    # update_csv(csv_file, predicted_species)

    if predicted_species:
        print(f"New audio is predicted to belong to the species with ID: {predicted_species}.")
    else:
        print("No prediction available.")

    print("Total time:", time.time() - start_time)

if __name__ == "__main__":
    main()


Train Accuracy: 1.0
Test Accuracy: 0.0
[310]
New audio is predicted to belong to the species with ID: 310.
Total time: 0.1801011562347412
